### Sentiment Analysis Related To The Increasing Amount of Indonesian VAT (PPN) to 12%.

#### 1. Business Understanding

Indonesian Govt wants to know the impact of the increasing amount of VAT from the headlines of online news outlets in Indonesia. The govt contacted a company (PT AI Bahagia) to classify the responds whether it is good or not.

#### 2. Objective Metrics

As a data analyst + scientist in the company, you are being tasked to create sentiment analysis from online news outlets in Indonesia.

#### 3. Dataset Preparation

The dataset was created by scraping Indonesian online news outlets.

(check scraper.ipynb)

Concating the data

#### 4. Data Labeling